# Session-by-session generalized modeling for participant 0 - Arousal 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn import metrics
from itertools import combinations
import itertools
import json
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn import neighbors
from sklearn.preprocessing import StandardScaler 
from sklearn.neural_network import MLPClassifier
pd.options.mode.chained_assignment = None
from sklearn.naive_bayes import GaussianNB
from imblearn.over_sampling import SVMSMOTE
from imblearn.over_sampling import BorderlineSMOTE
from crosstrainer import CrossTrainer
import numpy as np
from scipy import sparse
import scipy

## Read in dataset


In [ ]:
data0 = pd.read_csv("p0_THRI.csv")
data1 = pd.read_csv("p1_THRI.csv")
data2 = pd.read_csv("p2_THRI.csv")
data3 = pd.read_csv("p3_THRI.csv")

### Create a list of all participants' data

In [ ]:
data = [data0, data1, data2, data3]

### Display the main columns 

In [ ]:
basic_cols = []
for i in data[0].columns:
    if 'of_' not in i and 'op_' not in i and 'ros_' not in i and 'a_' not in i:
        basic_cols.append(i)
        
basic_cols = sorted(basic_cols)
print(len(basic_cols))

for i in basic_cols:
    print(i)

### Display the OpenFace and OpenPose features

In [ ]:
# Open Face Columns

of_cols = []
for i in data[0].columns:
    if ('of_' in i or 'op_' in i) and '_change' not in i and '_var' not in i:
        of_cols.append(i)
        
of_cols = sorted(of_cols)
print(len(of_cols))
for i in of_cols:
    print("'"+i+"',")

### Display the Audio features

In [ ]:
# Audio Columns

a_cols = []
for i in data[0].columns:
    if 'a_' in i and '_change' not in i and '_var' not in i:
        a_cols.append(i)
        
a_cols = sorted(a_cols)
print(len(a_cols))
for i in a_cols:
    print(i)

### Display the used performance ROS features

In [ ]:
# ROS Columns

ros_cols = []
for i in data[0].columns:
    if 'ros_' in i and '_change' not in i and '_var' not in i:
        ros_cols.append(i)
        
ros_cols = sorted(ros_cols)
print(len(ros_cols))

for i in ros_cols:
    print(i)

### Display the generated features from the windowing

1) Continuous features will have generated features named as "feature_var"

2) Discrete features will have generated features named as "feature_change"

In [ ]:
# For Window Only:
non_window_features = []
window_features = []
for i in data[0].columns:
    if i not in basic_cols:
        if 'change' in i or 'var' in i:
            window_features.append(i)
        else:
            non_window_features.append(i)
        
window_features = sorted(window_features)
print(len(window_features))
for i in window_features:
    print(i)

### Display the features where NaNs filled with max value

If openFace's or openPose's features are shown as NaN, there is no certain facial or body features are shown in the video frame. In this case, the value could be approximated by using the maximum value of the feature after standardization.

In [ ]:
# Columns where NaNs filled with max value
nan_max_cols = ['of_gaze_0_x',
'of_gaze_0_y',
'of_gaze_0_z',
'of_gaze_1_x',
'of_gaze_1_y',
'of_gaze_1_z',
'of_gaze_angle_x',
'of_gaze_angle_y',
'of_gaze_distance',
'of_gaze_distance_x',
'of_gaze_distance_y',
'of_pose_Rxv',
'of_pose_Ry',
'of_pose_Rz',
'of_pose_Tx',
'of_pose_Ty',
'of_pose_Tz',
'of_pose_distance',
'op_pose_c0',
'op_pose_c1',
'op_pose_c10',
'op_pose_c11',
'op_pose_c12',
'op_pose_c13',
'op_pose_c14',
'op_pose_c15',
'op_pose_c16',
'op_pose_c17',
'op_pose_c18',
'op_pose_c19',
'op_pose_c2',
'op_pose_c20',
'op_pose_c21',
'op_pose_c22',
'op_pose_c23',
'op_pose_c24',
'op_pose_c3',
'op_pose_c4',
'op_pose_c5',
'op_pose_c6',
'op_pose_c7',
'op_pose_c8',
'op_pose_c9',
'op_pose_x0',
'op_pose_x1',
'op_pose_x10',
'op_pose_x11',
'op_pose_x12',
'op_pose_x13',
'op_pose_x14',
'op_pose_x15',
'op_pose_x16',
'op_pose_x17',
'op_pose_x18',
'op_pose_x19',
'op_pose_x2',
'op_pose_x20',
'op_pose_x21',
'op_pose_x22',
'op_pose_x23',
'op_pose_x24',
'op_pose_x3',
'op_pose_x4',
'op_pose_x5',
'op_pose_x6',
'op_pose_x7',
'op_pose_x8',
'op_pose_x9',
'op_pose_y0',
'op_pose_y1',
'op_pose_y10',
'op_pose_y11',
'op_pose_y12',
'op_pose_y13',
'op_pose_y14',
'op_pose_y15',
'op_pose_y16',
'op_pose_y17',
'op_pose_y18',
'op_pose_y19',
'op_pose_y2',
'op_pose_y20',
'op_pose_y21',
'op_pose_y22',
'op_pose_y23',
'op_pose_y24',
'op_pose_y3',
'op_pose_y4',
'op_pose_y5',
'op_pose_y6',
'op_pose_y7',
'op_pose_y8',
'op_pose_y9']

###  The main loop for the session-by-session generalized modeling

In [ ]:
# The index of the testing participant
test_participant_index = 0

# Initialize the testing participant's data
test_participant_data = data[test_participant_index]

# Initialize a index list for training participants
train_index_list = [0,1,2,3]
train_index_list.remove(test_participant_index)

# Initialize the testing participant's list of session numbers
test_session_list = [1.0,2.0,3.0,4.0,5.0]

# Initialize a counter for the current testing session
current_session_counter = 1

# Initialize the size of the testing participant's sessions
size = len(test_session_list)

all_scores = []
all_y_test = []
all_preds = []

# Session-by-session generalized modeling
while(current_session_counter < size):
    
    # Initialize a list for each kind of training data
    # - train_data_list for all combinded data including both the early personal data from the test participant and data from other participants
    # - source_list for the data from other participants
    # - target_list for the early personal data from the test participant
    train_data_list = []
    source_list = []
    target_list = []
    
    # Append all the training participant's data into the lists
    for num in train_index_list:
        train_data_list.append(data[num].copy())
        source_list.append(data[num].copy())


    # Initialize the test_data dataframe to hold the testing datset
    test_data = pd.DataFrame()
    
    # A seperator for visual purpose of printed output
    print('-')
    
    # Append the adapted session's data into the train_data_list
    for j in range(current_session_counter):
        print("train: ", test_session_list[j])
        train_data_list.append(pd.DataFrame(test_participant_data.loc[test_participant_data['session_num'] == test_session_list[j]].copy()))
        target_list.append(pd.DataFrame(test_participant_data.loc[test_participant_data['session_num'] == test_session_list[j]].copy()))
        
    print('test:', test_session_list[current_session_counter])
    # Assign the testing session of the testing participant to the test_data
    test_data = test_participant_data.loc[test_participant_data['session_num'] == test_session_list[current_session_counter]].copy()
    
    # Concat all the training dataset into one dataframe
    train_data = pd.concat(train_data_list, ignore_index=True, sort=True)
    source_data = pd.concat(source_list, ignore_index=True, sort=True)
    target_data = pd.concat(target_list, ignore_index=True, sort=True)

    # Seperate out the features and the labels for training and testing set 
    y_train = train_data['C_Arousal']
    X_train = train_data.drop(columns=['C_Arousal', 'session_num', 'participant'])
    y_test = test_data['C_Arousal']
    X_test = test_data.drop(columns=['C_Arousal', 'session_num', 'participant']) 
    
    y_source = source_data['C_Arousal']
    X_source = source_data.drop(columns=['C_Arousal', 'session_num', 'participant'])
    y_target = target_data['C_Arousal']
    X_target = target_data.drop(columns=['C_Arousal', 'session_num', 'participant'])
    
     # Perform NaN standarlization and fill the NaN with corresponding values
    for c in X_train.columns:

        # Calculate the mean and std only using the data from other participants
        mean = np.nanmean(X_source[c])
        std = np.nanstd(X_source[c])
        
        if std == 0:
            X_train[c] = (X_train[c]-mean)
            X_test[c] = (X_test[c]-mean)
            X_source[c] = (X_source[c]-mean)
            X_target[c] = (X_target[c]-mean)
        else:
            X_train[c] = (X_train[c]-mean)/(std)
            X_test[c] = (X_test[c]-mean)/(std)
            X_source[c] = (X_source[c]-mean)/std
            X_target[c] = (X_target[c]-mean)/std

        # Fill in min_val for the generated values if NaN
        if c not in nan_max_cols:
            min_val = np.nanmin(X_source[c])
            X_train[c] = X_train[c].fillna(min_val)
            X_test[c] = X_test[c].fillna(min_val)
            X_source[c] = X_source[c].fillna(min_val)
            X_target[c] = X_target[c].fillna(min_val)
        # Fill in max_val for the openFace or OpenPose features if NaN
        else:
            max_val = np.nanmax(X_source[c])

            X_train[c] = X_train[c].fillna(max_val)
            X_test[c] = X_test[c].fillna(max_val) 
            X_source[c] = X_source[c].fillna(max_val)
            X_target[c] = X_target[c].fillna(max_val)

# ---------------------Model goes here-----------------------------------------

    model = XGBClassifier(n_estimators=100, max_depth = 6, booster = 'gbtree')
                    
# ------------------------------------------------------------------------------

    # Display the shape of the training and testing dataframe for each adaptation
    print(X_train.shape)
    print(X_source.shape)
    print(X_target.shape)
    print(X_test.shape)
    

    # Train the model with only the data from other participants
    model.fit(X_source,y_source)

    # Obtain the testing results
    scores = model.predict(X_test)
    probs = model.predict_proba(X_test)
    preds = probs[:,1]
    preds = pd.DataFrame(preds)
    scores = pd.DataFrame(scores)
    
    all_preds.append(preds)
    all_scores.append(scores)
    all_y_test.append(y_test)
    
    # Move the counter one index forward
    current_session_counter += 1



In [ ]:
df_all_scores = pd.concat(all_scores, ignore_index=True)
df_all_y_test = pd.concat(all_y_test, ignore_index=True)
df_all_preds = pd.concat(all_preds,ignore_index=True)

In [ ]:
avg_accuracy = accuracy_score(df_all_y_test, df_all_scores)
avg_auc = roc_auc_score(df_all_y_test, df_all_preds)
avg_precision = metrics.precision_score(df_all_y_test, df_all_scores, average=None)
avg_recall = metrics.recall_score(df_all_y_test, df_all_scores, average=None)
avg_f1 = metrics.f1_score(df_all_y_test, df_all_scores, average=None)
print(avg_auc)
print(avg_accuracy)
print(avg_f1)
print(avg_precision)
print(avg_recall)

In [ ]:
pd.DataFrame(df_all_scores).to_csv("p0_gen_xgb_arousal_all_scores.csv",index = False)
pd.DataFrame(df_all_y_test).to_csv("p0_gen_xgb_arousal_y_test.csv",index = False)
pd.DataFrame(df_all_preds).to_csv("p0_gen_xgb_arousal_all_preds.csv",index = False)